<a href="https://colab.research.google.com/github/kchenTTP/python-series/blob/main/intermediate_python_functions/Intermediate_Python_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Intermediate Python Functions**

In our fundamentals class, we learned about how to reuse code by defining functions. In this session, we'll deepen our understanding of python functions by covering some more features and best practices.


### **Table of Contents**

- [Default Parameters](#scrollTo=K3MSR-8x31WE)
- [Variable-Length Arguments](#scrollTo=Fkz86bj94ARo)
- [Function Scopes](#scrollTo=S5NweR1V40L9)
- [Argument Passing](#scrollTo=4Zk7BOGEbGBj)
- [Type Hints & Function Documentation](#scrollTo=nQ18LxcjJH90)


## **Default Parameters**

Default parameters allow you to define functions where some arguments have predetermined values if not specified by the caller. This makes functions more flexible and easier to use. Here's what you need to know about default parameters:

- Default parameters must come after non-default parameters
- Default values are evaluated when the function is defined, not when it's called
- Mutable default arguments (such as a list or dictionary) can cause unexpected behavior


In [29]:
# Default parameter after required parameter

def greet(name, greeting="Hello"):
  return f"{greeting}, {name}!"

In [30]:
# Error: Default parameter before required parameter

def bad_greet(greeting="Hello", name):  # This will cause a SyntaxError
  return f"{greeting}, {name}!"

SyntaxError: non-default argument follows default argument (<ipython-input-30-708ba64091d4>, line 3)

💡 When using a function with default parameters, the parameters that have default values are optional. If you choose to pass an argument for these parameters, the provided value will override the default one.

In [45]:
# Call function with default value
print(greet("Alice"))

# Override default value
print(greet("Bob", "Hi"))
print(greet("Tim", "G'day"))

Hello, Alice!
Hi, Bob!
G'day, Tim!


### **Common Pitfall: Mutable Default Arguments**

When using mutable default arguments, such as *lists* or *dictionaries*. If you modify the mutable object within the function, that change will persist across subsequent calls to the function. This can lead to unexpected behavior, as the default argument will retain its modified state instead of reverting to the initial value.

> 💡 To understand why, look at the [Argument Passing](#scrollTo=4Zk7BOGEbGBj) section below.

Take a look at the example below:

In [46]:
def duplicated_values(lst, duplicated=[], seen=[]):
  """Find duplicated values and keep track of all the values in a list."""

  for item in lst:
    if item in seen:
      duplicated.append(item)
      continue
    seen.append(item)
  return {
      "duplicated": duplicated,
      "seen": seen
  }

In [47]:
# First call
print(duplicated_values([1, 2, 3, 2]))

# Second call
print(duplicated_values([4, 4, 5, 6]))

{'duplicated': [2], 'seen': [1, 2, 3]}
{'duplicated': [2, 4], 'seen': [1, 2, 3, 4, 5, 6]}


As you can see, in our second function call, even though we intended to create a new list with different values, the values from our first function call still appeared.

This is because the mutable default argument (the list) retains any changes made to it across all calls to the function. Instead of starting fresh with each call, it continues using the modified list from previous calls.

<br>

To avoid this pitfall, it's recommended to use `None` as a default value and then create a new mutable object inside the function if needed.

In [48]:
def duplicated_values_fixed(lst, duplicated = None, seen = None):
  """Find duplicated values and keep track of all the values in a list."""

  if duplicated is None:
    duplicated = []
  if seen is None:
    seen = []

  for item in lst:
    if item in seen:
      duplicated.append(item)
      continue
    seen.append(item)
  return {
      "duplicated": duplicated,
      "seen": seen
  }

In [49]:
# First call
print(duplicated_values_fixed([1, 2, 3, 2]))

# Second call
print(duplicated_values_fixed([4, 4, 5, 6]))

{'duplicated': [2], 'seen': [1, 2, 3]}
{'duplicated': [4], 'seen': [4, 5, 6]}


This approach ensures that each call to `duplicated_values_fixed()` creates a new list if one isn't provided, keeping the function's behavior consistent and free from unintended side effects.

### **Exercise**

Create a function called `format_price()` that formats a price with optional currency and decimal places. Make sure to provide a default value for the parameters so that the function can still be used without explicitly passing them in.

- Currency symbol example: $, €, ¥, £
- Decimal places:
  - `2`: 0.12
  - `3`: 1.356


In [ ]:
# Your code Goes here


In [ ]:
# @title Solution

def format_price(price, currency="$", decimal_places=2):
  return f"{currency}{price:.{decimal_places}f}"


print(format_price(24.99))
print(format_price(24.99, "€"))
print(format_price(24.99, "¥", 0))

## **Variable-Length Arguments**

Python provides two special parameters for handling variable numbers of arguments: `*args` and `**kwargs`.

- `*args`: Collects any number of **positional** arguments into a ***tuple***.
- `**kwargs`: Collects any number of **keyword** arguments into a ***dictionary***.

> ❗**IMPORTANT:** The order of argument matters:
>
>  `regular args` → `*args` → `default args` → `**kwargs`


In [50]:
# Argument ordering
def example_function(required, *args, default="default", **kwargs):
  print(f"Required: {required}")
  print(f"Args: {args}")
  print(f"Default: {default}")
  print(f"Kwargs: {kwargs}")

Positional arguments must come before keyword arguments in the function call, or you'll encounter an error.


In [2]:
example_function("required", 1, 2, 3, default="custom", extra1="value1", extra2="value2")

Required: required
Args: (1, 2, 3)
Default: custom
Kwargs: {'extra1': 'value1', 'extra2': 'value2'}


🚨 Typically, when you pass arguments with their parameter names, like `func(arg1=value)`, the order doesn't matter because each argument is explicitly tied to its parameter. However, if you mix positional arguments (`*args`) and keyword arguments (`**kwargs`), order becomes important.


Inside the function, you can access `*args` using indices, as it is stored as a tuple, and you can access `**kwargs` using dictionary keys or methods like `.get()` or `.items()` since it's stored as a dictionary.


In [3]:
# Accessing args and kwargs
def example_function_2(*args, **kwargs):
    # Accessing args
    print(args[0])  # Access the first positional argument

    # Accessing kwargs
    print(kwargs.get('name'))  # Access the 'name' keyword argument if it exists
    print(kwargs.items())  # Get all key-value pairs as a list of tuples

    if 'age' in kwargs:
        print(kwargs['age'])  # Access the 'age' keyword argument if it exists

In [4]:
example_function_2(1, 2, 3, name="Alice", age=25)

1
Alice
dict_items([('name', 'Alice'), ('age', 25)])
25


Arguments can also be passed into a funciton in a list (if `*args`) or a dictionary (if `**kwargs`) while being **unpacked** like this:


In [5]:
arg_list = ["apple", "banana", "kiwi"]
kwarg_dict = {"name": "Alice", "age": 25}

example_function_2(*arg_list, **kwarg_dict)

apple
Alice
dict_items([('name', 'Alice'), ('age', 25)])
25


- `*arg_list` when passing a list as argument is called list unpacking
- `**kwarg_dict` when passing a dictionary as argument is called dictionary unpacking


> 📒 **NOTE:** The convention is to use `args` and `kwargs`, but any valid variable name works


In [1]:
def sum(*numbers):
  total = 0
  for num in numbers:
    total += num
  return total

print(sum(1, 2, 3))
print(sum(4, 5, 6, 7))

6
22


### **Keyword-Only Arguments**

You can use `*` to enforce that any parameter defined after `*` must be passed as a keyword argument.

This means you must specify the parameter name when passing arguments to these parameters; otherwise, you'll get an error.

> 📒 **Note**: This approach is especially useful for improving code readability and preventing accidental errors in function calls.


In [14]:
def greet_modified(name, *, greeting="Hello"):
  return f"{greeting}, {name}!"

print(greet_modified("Alice"))
print(greet_modified("Bob", greeting="Hi"))

Hello, Alice!
Hi, Bob!


In [15]:
# Not allowed: Didn't provide parameter name to keyword-only argument
print(greet_modified("Tom", "Morning"))

TypeError: greet_modified() takes 1 positional argument but 2 were given

In [16]:
# Not allowed: Arguments in incorrect order
print(greet_modified(name="Tom", "Morning"))

SyntaxError: positional argument follows keyword argument (<ipython-input-16-19dfc297c491>, line 2)

### **Exercise: Generate API Endpoints**

Many APIs allow you to pass multiple parameters to customize requests.

Let's write a function that takes:
1. a base API endpoint as argument
2. your API key as argument
3. a variable number of API parameters as keyword arguments

This function will then generate a complete API URL with the parameters included.

> Hint: Use `?` after the base API endpoint before passing in the parameters and use `&` for multiple API parameters
>
> Example: `https://www.google.com/search?q=apple&sourceid=chrome&ie=UTF-8`



In [9]:
example_endpoint = "https://example.com/api"
api_key = "my_api_key_123"
params = {"format": "json", "limit": 10}

In [ ]:
# Your code goes here


In [ ]:
# @title Solution

def generate_api_endpoint(base_url, api_key, **params):
  url = f"{base_url}?api_key={api_key}"
  for key, value in params.items():
    url += f"&{key}={value}"
  return url

# Test solution
url = generate_api_endpoint(example_endpoint, api_key, format="json", limit=10)
print(url)
url = generate_api_endpoint(example_endpoint, api_key, **params)
print(url)

## **Function Scopes**

Function scopes define the accessibility and ***lifespan of variables*** within different parts of a program. Understanding scopes help prevent variable conflicts and ensures that functions work as intended by controlling where and how variables can be accessed. Here's what you need to know:

- **Local Scope**: Variables defined within a function are local to that function and can't be accessed outside of it.
- **Global Scope**: Variables defined outside any function are in the global scope and can be accessed from anywhere in the program, including inside functions (unless there's a variable inside the function that shares the same name).
- **Nonlocal Scope**: Variables in a nested (enclosing) function can be accessed and modified in an inner function using the nonlocal keyword.

> 📒 **The LEGB Rule**: Python follows the Local, Enclosing, Global, Built-in (LEGB) rule to determine the order of scope resolution.


### **Local Scope**

Variables defined within a function are only accessible within that function. They cannot be accessed from outside unless explicitly returned.


In [12]:
# Accessing local variable outside of function
def foo():
  # local variable
  x = 50
  print(f"Local: {x = }")

foo()
print(f"Global: {x = }")

Local: x = 50


NameError: name 'x' is not defined

As you can see you'll get a `NameError` because Python cannot access the local variable `x`.

> 💡 In order to access the variables within a function outside the function itself, you need to explicitly return it to the outside.


In [13]:
# Return local variable outside the function
def bar():
  y = 300
  return y

result = bar()
print(f"Global: {result = }")

Global: result = 300


### **Global Scope**

Variables defined outside any function are in the `global` scope and can be accessed from anywhere in the program.


In [14]:
# Accessing global variables within a function
z = 200

def baz():
  print(f"Global: {z = }")

baz()

Global: z = 200


> 🚨 You are, however, not allowed to reassign global variables within a function. Doing so will result in a new local variable with the same name being created.

In [15]:
# Reassigning global variables within a function
w = "Hello"

def qux():
  # Attempting to reassing new value
  w = "World"
  print(f"Local: {w = }")

qux()
print(f"Global: {w = }")

Local: w = 'World'
Global: w = 'Hello'


As you can see, we get two different values for `w`. This happens because one value is from the global variable, while the other is from within the function's local scope.

If you need to modify a global variable inside a function, it's generally recommended to either:

1. Pass the variable as an argument, modify it within the function, and then return the updated value.
2. Use the `global` keyword to explicitly declare that you're modifying the global variable.


In [16]:
g = 100

# Passing global variable as argument
def quux(g):
  g = 50
  print(f"Local: {g = }")
  return g

# Reassigning new value to variable
g = quux(g)
print(f"Global: {g = }")

Local: g = 50
Global: g = 50


In [17]:
# Using the global keyword
h = 100

def quuz():
  # Specify we're using the global variable and not defining a local one
  global h
  h = 50
  print(f"Local: {h = }")

quuz()
print(f"Global: {h = }")

Local: h = 50
Global: h = 50


### **Nonlocal (Enclosing) Scope**

Nonlocal, or enclosing scope, refers to variables defined in a parent function that can be accessed and modified within a nested (inner) function. The `nonlocal` keyword allows you to work with these variables, enabling the inner function to modify the state of variables in its enclosing function.

- **Nested Functions**: functions defined inside other functions.
- **The `nonlocal` Keyword**: This keyword allows an inner function to modify a variable from the outer (enclosing) function's scope, rather than creating a new local variable.
  
> 🚨 Using nonlocal scope can be useful for managing state across nested functions without relying on global variables, but should be used thoughtfully to keep the code easy to follow.


In [18]:
# Nested functions
def outer():
  k = "outer"
  def inner():
    k = "inner"
    print(f"Inner function: {k = }")

  inner()
  print(f"Outer function: {k = }")

outer()

Inner function: k = 'inner'
Outer function: k = 'outer'


In [19]:
# Using the nonlocal keyword
def outer2():
  l = "outer"
  def inner():
    # Specify we're using the variable from the outer function and not defining a local one
    nonlocal l
    l = "changed by inner"
    print(f"Inner function: {l = }")

  inner()
  print(f"Outer function: {l = }")

outer2()

Inner function: l = 'changed by inner'
Outer function: l = 'changed by inner'


## **Argument Passing in Python**

Python passes arguments to functions **by object reference** (often called "pass-by-assignment"). This means that:

- `Pass by Value`: For **immutable types** (e.g., integers, strings, tuples), the function receives *a copy* of the reference, and the original object remains *unchanged* if modified inside the function.
- `Pass by Reference`: For **mutable types** (e.g., lists, dictionaries), the function receives a reference to the same object, so any in-place modifications affect the original object outside the function.


In [30]:
def modify_value(value):
    # Local scope for modify_value
    value += 5  # This modifies only the local copy for immutables
    return value

# Immutable argument passing
num = 20
print(f"Before modify_value: {num = }")
num_2 = modify_value(num)
print(f"After modify_value: {num = }")

Before modify_value: num = 20
After modify_value: num = 20


💡 You can see in the example below, the function is modifying the same list, even though it looks like we are passing the list of `[1, 2, 3]` in every time we call the function.


In [20]:
def modify_list(my_list):
    # Local scope for modify_list
    my_list.append(4)  # This modifies the original list (a mutable object)

# Mutable argument passing
numbers = [1, 2, 3]
print(f"Before modify_list: {numbers = }")
modify_list(numbers)
print(f"After modify_list: {numbers = }")

Before modify_list: numbers = [1, 2, 3]
After modify_list: numbers = [1, 2, 3, 4]


## **Type Hints & Function Documentation**


### **Type Hints**

Type hints provide a way to annotate code with expected types.

<br>

**Benefits include:**

- **Self-documenting code**
  
  Clarifies what types are expected, making code easier to read.

- **IDE support and code completion**

  Enhances code suggestions and error detection in many code editors.

- **Static type checking**

  Tools like [mypy](https://mypy.readthedocs.io/en/stable/index.html) or [pyright](https://github.com/microsoft/pyright) can catch type-related errors before runtime.

- **Ease of understanding and refactoring**

  Simplifies the process of understanding code, especially in larger projects.

<br>

> 📒 **Note**: Type hints do not change the functionality of your code and are completely optional. However, including them can significantly improve the readability and maintainability of your code.


Let's look at the difference between a function with type hints and a function without type hints:


In [31]:
# No type hint example
def cube_(number):
  return number ** 3

number = input("Enter a number: ") # input() function returns a string

# We don't get a red squiggly line from the code editor because it doesn't know what data type to expect even though this will clearly error
print(cube_(number))

Enter a number: two


TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'

In [33]:
# Basic type hint example
def process_string(text: str, times: int = 1) -> str:
  return text * times

def cube(number: float) -> float:
  return number ** 3

number = input("Enter a number: ") # input() function returns a string

# We get a red squiggly line from the code editor because this will raise an error: strings cannot be used in arithmetic operations.
print(cube(number))

Enter a number: two


TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'

We can even write type hints for variables

> 🚨 There is no need to do this for every variable but for obscure ones where the datatype of the variable is very hard to infer, this is extremely useful!


In [21]:
# Type hinting variables
num_1: int = 1
num_2: float = 1.0

var_3 = {"a": 1, "b": 2, "c": 3, "d": 4}
num_3: int | None = var_3.get("apple") # in this example get() can either return an 'int' or 'None'. Using type hint makes it clearer to see what kind of datatype you are getting

If our function or variable can take or return multiple data types, we can use the `|` operator to separate multiple types:


In [22]:
# Multiple acceptable types
def is_even(num: int | float) -> bool:
  return num % 2 == 0

Here are some more type hint examples:


In [25]:
class CustomType:
  def do_sth(self) -> None:
    pass

# Using our own class for type hint
def construct_custom_type() -> CustomType:
  return CustomType()

In [26]:
from typing import Literal

# Parameter `color` expects string literals: "red", "green", or "blue"
def choose_color(color: Literal["red", "green", "blue"]) -> None:
  if not isinstance(color, str): raise TypeError("Expected a string")
  if not color.lower() in ["red", "green", "blue"]: raise ValueError("Invalid color")

  match color:
    case "red":
      pass
    case "green":
      pass
    case "blue":
      pass
    case _:
      print("Shouldn't be able to reach here, something is very wrong!")
  print(f"You chose {color}")

choose_color("red")
choose_color("green")
choose_color("blue")

You chose red
You chose green
You chose blue


In [34]:
from collections.abc import Callable
from typing import Any

# Parameter `func` expects a function or method. `args` and `kwargs` can be any data type
def repeat_func(func: Callable, times: int, *args: Any, **kwargs: Any) -> None:
  for _ in range(times):
    rv = func(*args, **kwargs)
    print(rv)


repeat_func(greet, times=2, name="Timmy", greeting="What's up")
repeat_func(cube, times=3, number=2)

What's up, Timmy!
What's up, Timmy!
8
8
8


In [36]:
import pandas as pd

# Using data types from some python package
def create_df(data: dict) -> pd.DataFrame:
  return pd.DataFrame(data)


data = {
    'name': ['Alice', 'Bob', 'Charlie'],
    'age': [25, 30, 35],
    'city': ['New York', 'London', 'Paris']
}

df = create_df(data)
df

,name,age,city
0,Alice,25,New York
1,Bob,30,London
2,Charlie,35,Paris


For more in-depth information on writing type hints, check out [Python's documentation](https://peps.python.org/pep-0484/)


### **Docstrings**

Docstrings provide a standardized way to document Python functions and can serve multiple purposes, including:

- Clear documentation for other developers
- Interactive help, such as through the `help()` function in Python or hover text in code editors
- Source for automatic documentation generators like [Sphinx](https://www.sphinx-doc.org/en/master/index.html) and [Swagger](https://swagger.io/docs/).

A well-structured docstrings can improve code readability and make maintenance a lot easier. Here are a few popular docstring styles to consider:

<br>

**Common Docstring Styles**

- [Google Style](https://github.com/google/styleguide/blob/gh-pages/pyguide.md#38-comments-and-docstrings): Known for its readability, with parameters and return values clearly outlined.
- [NumPy Style](https://numpydoc.readthedocs.io/en/latest/format.html#docstring-standard): Particularly useful in scientific and data-centric projects, with structured sections for extended explanations.
- [reStructuredText Style](https://sphinx-rtd-tutorial.readthedocs.io/en/latest/docstrings.html): Often used with Sphinx, this style supports extensive formatting options and compatibility with reStructuredText-based documentation tools.


In [37]:
# Google Style Example
def scream(some_string: str) -> None:
  """Scream out the input string.

  Converts all characters in a string to uppercase to simulate screaming. The longer the string, the more exclamation marks will be added at the end.

  Args:
    some_string (str): The string to convert to uppercase.

  Returns:
    None

  Raises:
    TypeError: If the input is not a string.
    ValueError: If the input is an empty string.

  Examples:
    >>> scream("go home denis, you are drunk")
    GO HOME DENIS, YOU ARE DRUNK!!!!!!!!!!!!!!
  """
  if not isinstance(some_string, str):
    raise TypeError(f"Expected a string, instead got: {type(some_string)}")

  some_string = some_string.strip()

  if not some_string:
    raise ValueError("Expected a non-empty string")

  volume = max(len(some_string) // 2, 1)
  print(f"{some_string.upper()}{'!' * volume}")

Now, if you use the `help()` function or hover your mouse over the function name in the code cell or an editor, the tooltip will show the function signature along with the docstring.

This makes it much easier to quickly understand what the function does, its parameters, and any exceptions it may raise.


In [38]:
help(scream)

Help on function scream in module __main__:

scream(some_string: str) -> None
    Scream out the input string.
    
    Converts all characters in a string to uppercase to simulate screaming. The longer the string, the more exclamation marks will be added at the end.
    
    Args:
      some_string (str): The string to convert to uppercase.
    
    Returns:
      None
    
    Raises:
      TypeError: If the input is not a string.
      ValueError: If the input is an empty string.
    
    Examples:
      >>> scream("go home denis, you are drunk")
      GO HOME DENIS, YOU ARE DRUNK!!!!!!!!!!!!!!



In [39]:
# Try hovering your mouse over the the function name

scream("Shhh")
scream("It's 10 past 12 already")
scream("Be quiet")

SHHH!!
IT'S 10 PAST 12 ALREADY!!!!!!!!!!!
BE QUIET!!!!


For more in-depth information on writing effective docstrings, check out this [article on docstrings](https://www.dataquest.io/blog/documenting-in-python-with-docstrings/).


## **Conclusion**

That's all for our intermediate Python functions class! If you want to dive deeper into the topics we covered, check out these resources:

### **Additional Online Resources**

- **[Python's Official Documentation on Functions](https://docs.python.org/3/tutorial/controlflow.html#defining-functions)**
- **[W3Schools - Python Functions](https://www.w3schools.com/python/python_functions.asp)**
- **[Real Python - Python Functions Tutorial](https://realpython.com/defining-your-own-python-function/)**
- **[GeeksforGeeks - Python Functions](https://www.geeksforgeeks.org/python-functions/)**

<br>

If you're craving to learn even more about Python functions, join our [advanced functions](https://colab.research.google.com/drive/1jV_L1nTbk2aEP3a2Il8smEcyzs20a2j3?usp=sharing) class! We'll dive into **lambda functions, decorators, recursion,** and more. Looking forward to seeing you there! 🚀
